In [1]:
import pandas as pd

In [109]:
pos = pd.read_csv('positive.csv', delimiter=';', header=None)[[3]]
neg = pd.read_csv('negative.csv', delimiter=';', header=None)[[3]]

pos = pos.rename(columns={3: 'Text'})
neg = neg.rename(columns={3: 'Text'})

In [110]:
# 1 - положительная эмоция
# -1 - отрицательная эмоция

pos['Emotion'] = 1
neg['Emotion'] = -1

In [111]:
frames=[pos,neg]
df=pd.concat(frames).sample(frac=1).dropna().drop_duplicates()

In [112]:
df['Text'] = df.apply(lambda row : row['Text'].lower(), axis = 1) 

In [113]:
df.head()

,Text,Emotion
51531,нужна ли тебе ёлка на новый год? — да) http://...,1
11120,женщины на распродажах носятся с такими глазам...,1
47222,"rt @___alena: кто бы мог подумать,что все так ...",1
19150,"rt @jiley__belieber: @mbi_ctpemhbie эй бл, я в...",-1
8598,rt @sejivylofysy: а эпичненькую антенну мы сег...,1


In [114]:
import re 
import nltk
nltk.download('stopwords')
import  pymorphy2
morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Za-z0–9!#$%&\'\()*+,./:;?@[\]^_{|}~—\"\-]+"
stopwords_ru = nltk.corpus.stopwords.words('russian')
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    if len(tokens) > 2:
        return ''.join(tokens[x] if x == 0  else ' ' + tokens[x] for x in range(len(tokens)))
    return None

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/imrantimkanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
df[['Text']] = df['Text'].apply(lemmatize).to_frame()

In [121]:
df = df.dropna()

# V2

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 

X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Emotion'],
                                                    test_size=0.33, random_state=42)

In [137]:
estimators = []


vectorizer =  TfidfVectorizer()
classifier = LogisticRegression()


estimators.append(('vectorization', vectorizer))
estimators.append(('classification', classifier))


pipe = Pipeline(estimators)

In [138]:
params = {}
logreg_params = LogisticRegression().get_params()
    
grid_search = GridSearchCV(pipe, params)


In [139]:
grid_search.fit(X_train, y_train)

/Users/imrantimkanov/Desktop/all/jupyter/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/imrantimkanov/Desktop/all/jupyter/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

GridSearchCV(estimator=Pipeline(steps=[('vectorization', TfidfVectorizer()),
                                       ('classification',
                                        LogisticRegression())]),
             param_grid={})

In [140]:
grid_search.score(X_test, y_test)

0.7157307798975318

In [141]:
grid_search.predict(['плохо все да'])

array([-1])

In [142]:
import pickle

filename = 'finalized_model.sav'
pickle.dump(grid_search, open(filename, 'wb'))